In [8]:
from requests import get
from scipy.stats import ttest_rel
import numpy as np

metric = 'ndcg'
cutoff = '10'
n_questions = 5
experiment_subset = 'equal'
significance_level = 0.05
data = get(f'http://localhost:5000/spectate/results/{experiment_subset}/{metric}/{cutoff}').json()
de_only = False
recommenders = ['ppr-joint', 'knn', 'mf', 'ppr-collab', 'ppr-kg']

if not de_only:
    recommenders = [f'{recommender}-rec' for recommender in recommenders]

interviewer_a = 'greedy-adaptive'
interviewer_b = 'pop'

for recommender in recommenders:
    model_a = f'{interviewer_a}-{recommender}'
    model_b = f'{interviewer_b}-{recommender}'
    
    if model_a not in data or model_b not in data:
        continue
        
    data_a = data[model_a][n_questions - 1]
    data_b = data[model_b][n_questions - 1]
    
    t, p = ttest_rel(data_a, data_b)
    print(f'{"*" if p <= significance_level else ""}{recommender}: {p:.5f}')

def comp(a, b):
    data_a = data[a][n_questions - 1]
    data_b = data[b][n_questions - 1]
    
    t, p = ttest_rel(data_a, data_b)
    
    print(f'{a}: {np.mean(data_a):.4f} ({np.std(data_a):.2f})')
    print(f'{b}: {np.mean(data_b):.4f} ({np.std(data_b):.2f})')
    
    print(f'{"*" if p <= significance_level else ""}: {p:.5f}')

comp('greedy-adaptive-ppr-joint', 'greedy-adaptive-ppr-joint-knowdontknow')

greedy-adaptive-ppr-joint: 0.0974 (0.00)
greedy-adaptive-ppr-joint-knowdontknow: 0.0937 (0.00)
*: 0.02903
